In [ ]:
import pathlib
import pprint
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (
    core as oc,
    net_quantity as onq,
    unit_system as units,
)

In [ ]:
import option
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import Array, Int32

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
    'permian': 'Project_frankNstein_Permian_UTM13_FEET.ifrac'
}

In [ ]:
projects = toolz.pipe(
    project_filenames,
    toolz.valmap(lambda fn: str(pathlib.Path(r'c:\src\Orchid.IntegrationTestData').joinpath(fn))),
    toolz.valmap(toolz.curry(oc.load_project)),
)
pprint.pprint(projects)

In [ ]:
native_projects = toolz.valmap(lambda p: p.dom_object, projects)
pprint.pprint(native_projects)

In [ ]:
wells = toolz.pipe(native_projects,
    toolz.valmap(lambda native_project: native_project.Wells.Items),
    toolz.valmap(lambda well_map: [(w.Name, w) for w in well_map]),
    toolz.valmap(dict))
pprint.pprint(wells)

In [ ]:
trajectories = toolz.valmap(toolz.valmap(lambda w: w.Trajectory), wells)
pprint.pprint(trajectories)

In [ ]:
project_units = toolz.valmap(lambda p: p.project_units, projects)
pprint.pprint(project_units)

In [ ]:
project_name = 'bakken'

# well_name = 'Demo_1H'
# samples = [0, 252, 146, 174, 99, 185]
# well_name = 'Demo_2H'
# samples = [0, 245, 203, 154, 155, 241]
# well_name = 'Demo_3H'
# samples = [0, 233, 121, 19, 213, 53]
well_name = 'Demo_4H'
samples = [0, 256, 53, 14, 144, 140]

In [ ]:
eastings = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetEastingArray(WellReferenceFrameXy.Project),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(eastings)

In [ ]:
northings = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetNorthingArray(WellReferenceFrameXy.Project),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(northings)

In [ ]:
tvdSss = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetTvdArray(DepthDatum.SeaLevel),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(tvdSss)

In [ ]:
inclinations = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetInclinationArray(),
    toolz.map(lambda ne: onq.as_measurement(units.Common.ANGLE, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(inclinations)

In [ ]:
azimuths = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetAzimuthEastOfNorthArray(),
    toolz.map(lambda ne: onq.as_measurement(units.Common.ANGLE, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(azimuths)

In [ ]:
mdKbs = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetMdKbArray(),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(mdKbs)

In [ ]:
frame_data = {
    'field': [project_name] * len(samples),
    'well': [well_name] * len(samples),
    'index': samples,
    'easting': eastings,
    'northing': northings,
    'tvdss': tvdSss,
    'inclination': inclinations,
    'azimuth': azimuths,
    'mdkb': mdKbs,
}

In [ ]:
pd.DataFrame(data=frame_data)